In [66]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import sys
import os
import logging
import datetime

import tensorflow as tf
from tensorflow import keras

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

In [67]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [1]:
data_dir = "/ssd003/projects/pets/datasets/home_credit"
# home_credit_train_df = pd.read_csv(f"{data_dir}/home_credit_train.csv")
# credit_bureau_train_df = pd.read_csv(f"{data_dir}/credit_bureau_train.csv")
data_df = pd.read_csv(f"{data_dir}/train.csv")

NameError: name 'pd' is not defined

In [ ]:
## data set prep

In [2]:
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

NameError: name 'train_test_split' is not defined

In [3]:
train_labels = np.array(train_df.pop('target'))
val_labels = np.array(val_df.pop('target'))
test_labels = np.array(test_df.pop('target'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

NameError: name 'np' is not defined

In [72]:
# normaliza data

In [73]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)

In [74]:
#oversampling
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

X_train_oversampled, y_oversampled = oversample.fit_resample(train_features, train_labels)

## data handling


In [75]:
TRAIN_PERC = .9
BATCH_SIZE = 1024
NUM_TEACHERS = 15
TEACHER_EPOCHS = 20
STUDENT_EPOCHS = 20
EPOCHS = 100

early_stopping = tf.keras.callbacks.EarlyStopping(
#     monitor='val_prc', 
    monitor="val_auc",
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [76]:
def get_loaders(data,lables, num_teachers, batch_size):
    """ 
    Function to create data loaders for the Teacher Class.
    
    :param data: Numpy Array of the data 
    :param num_teacher: Number of teacher models 
    :param batch_size: Batch size for the dataloaders

    :return: Return teacher loaders and student loader (with actual labels)
    """ 
    loaders = []
    train_loaders = []
    sample_size = len(data) // (num_teachers + 1)

    for i in range(num_teachers):
        indices = list(range(i*sample_size, (i+1)*sample_size))
        #subset_data = Subset(data, indices)
        subset_data = data[indices[0]:indices[-1]]
        #loader = DataLoader(subset_data, batch_size=batch_size)
        loaders.append(subset_data)
        train_loaders.append(lables[indices[0]:indices[-1]])
        
    return loaders[1:], loaders[0], train_loaders #teacher, students

In [77]:
# Define dataloaders

t_loaders, s_loader, resampled_labels = get_loaders(X_train_oversampled,y_oversampled, NUM_TEACHERS, BATCH_SIZE) # Teacher loaders, student loader
# val_loader = DataLoader(val_data, batch_size=BATCH_SIZE, drop_last=True) # Loader to validate in Train Ensemble and Train Student Model

In [78]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
    if output_bias is not None:
        output_bias = tf.keras.initializers.Constant(output_bias)
    # underfitting. 
    model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
    ])
    model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

    return model

In [79]:
# Initialize models and otptimizers for teacher ensembles
models = [make_model() for i in range(NUM_TEACHERS)]
# opts = [torch.optim.Adam(model.parameters(), lr=.001,  betas=(0.9, 0.999)) for model in models]


In [80]:
def train_model(resampled_model, t_loader, resampled_label):
    resampled_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=METRICS)

    # Reset the bias to zero, since this dataset is balanced.
    output_layer = resampled_model.layers[-1] 
    output_layer.bias.assign([0])
    # resampled_steps_per_epoch = 20

    resampled_history = resampled_model.fit(
        t_loader, resampled_label,
        epochs=EPOCHS,
    #     steps_per_epoch=resampled_steps_per_epoch,
        callbacks=[early_stopping],
        validation_data=(val_features, val_labels))

In [82]:
# train_model(models[0],t_loaders[0],resampled_labels[0]  )
for i in range(len(models)):
    print(i,'-------------------------------------')
    train_model(models[i],t_loaders[i],resampled_labels[i])

0 -------------------------------------
Epoch 1/100
707/707 [==============================] - 14s 17ms/step - loss: 0.3397 - tp: 9.0000 - fp: 47.0000 - tn: 29535.0000 - fn: 2597.0000 - accuracy: 0.9179 - precision: 0.1607 - recall: 0.0035 - auc: 0.5209 - prc: 0.0852 - val_loss: 0.2866 - val_tp: 1.0000 - val_fp: 4.0000 - val_tn: 45156.0000 - val_fn: 4041.0000 - val_accuracy: 0.9178 - val_precision: 0.2000 - val_recall: 2.4740e-04 - val_auc: 0.5484 - val_prc: 0.0909
Epoch 2/100
707/707 [==============================] - 10s 15ms/step - loss: 0.3084 - tp: 0.0000e+00 - fp: 1.0000 - tn: 20790.0000 - fn: 1829.0000 - accuracy: 0.9191 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5248 - prc: 0.0878 - val_loss: 0.2889 - val_tp: 0.0000e+00 - val_fp: 1.0000 - val_tn: 45159.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5016 - val_prc: 0.0805
Epoch 3/100
707/707 [==============================] - 10s 15ms/step - loss: 0.29

Epoch 2/100
707/707 [==============================] - 11s 15ms/step - loss: 0.3082 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 20731.0000 - fn: 1889.0000 - accuracy: 0.9165 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5216 - prc: 0.0885 - val_loss: 0.2894 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 45160.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4798 - val_prc: 0.0764
Epoch 3/100
707/707 [==============================] - 11s 15ms/step - loss: 0.3005 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 20731.0000 - fn: 1889.0000 - accuracy: 0.9165 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5293 - prc: 0.0927 - val_loss: 0.2869 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 45160.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5033 - val_prc: 0.0807
Epoch 4/100
707/707 [==============================] - 11s 15ms/step - loss: 0.3005 - 

Epoch 26/100
707/707 [==============================] - 11s 16ms/step - loss: 0.2748 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 20821.0000 - fn: 1799.0000 - accuracy: 0.9205 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5771 - prc: 0.1050 - val_loss: 0.2849 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 45160.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5156 - val_prc: 0.0835
Epoch 27/100
707/707 [==============================] - 11s 16ms/step - loss: 0.2747 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 20821.0000 - fn: 1799.0000 - accuracy: 0.9205 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5788 - prc: 0.1070 - val_loss: 0.2847 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 45160.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5143 - val_prc: 0.0836
Epoch 28/100
707/707 [==============================] - 11s 16ms/step - loss: 0.2739

Epoch 3/100
707/707 [==============================] - 11s 15ms/step - loss: 0.3047 - tp: 1.0000 - fp: 2.0000 - tn: 20796.0000 - fn: 1821.0000 - accuracy: 0.9194 - precision: 0.3333 - recall: 5.4885e-04 - auc: 0.5084 - prc: 0.0835 - val_loss: 0.2870 - val_tp: 0.0000e+00 - val_fp: 1.0000 - val_tn: 45159.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5194 - val_prc: 0.0846
Epoch 4/100
707/707 [==============================] - 11s 15ms/step - loss: 0.2966 - tp: 0.0000e+00 - fp: 2.0000 - tn: 20796.0000 - fn: 1822.0000 - accuracy: 0.9194 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5108 - prc: 0.0856 - val_loss: 0.2830 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 45160.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5581 - val_prc: 0.0971
Epoch 5/100
707/707 [==============================] - 11s 16ms/step - loss: 0.2915 - tp: 0.0000e+00 - fp:

Epoch 3/100
707/707 [==============================] - 11s 16ms/step - loss: 0.3032 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 20789.0000 - fn: 1831.0000 - accuracy: 0.9191 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5205 - prc: 0.0876 - val_loss: 0.2920 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 45160.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4555 - val_prc: 0.0724
Epoch 4/100
707/707 [==============================] - 11s 16ms/step - loss: 0.2963 - tp: 0.0000e+00 - fp: 0.0000e+00 - tn: 20789.0000 - fn: 1831.0000 - accuracy: 0.9191 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5191 - prc: 0.0866 - val_loss: 0.2902 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 45160.0000 - val_fn: 4042.0000 - val_accuracy: 0.9178 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.4565 - val_prc: 0.0724
Epoch 5/100
707/707 [==============================] - 15s 21ms/step - loss: 0.2894 - 

...

707/707 [==============================] - 11s 16ms/step - loss: 0.0389 - tp: 22598.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 22.0000 - accuracy: 0.9990 - precision: 1.0000 - recall: 0.9990 - auc: 0.0000e+00 - prc: 1.0000 - val_loss: 7.6053 - val_tp: 4042.0000 - val_fp: 45126.0000 - val_tn: 34.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.0828 - val_precision: 0.0822 - val_recall: 1.0000 - val_auc: 0.5749 - val_prc: 0.0960
Epoch 3/100
707/707 [==============================] - 12s 17ms/step - loss: 0.0192 - tp: 22616.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 4.0000 - accuracy: 0.9998 - precision: 1.0000 - recall: 0.9998 - auc: 0.0000e+00 - prc: 1.0000 - val_loss: 9.3783 - val_tp: 4042.0000 - val_fp: 45140.0000 - val_tn: 20.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.0826 - val_precision: 0.0822 - val_recall: 1.0000 - val_auc: 0.5390 - val_prc: 0.0886
Epoch 4/100
707/707 [==============================] - 12s 17ms/step - loss: 0.0128 - tp: 22619.0000 - fp: 0.0000e+00 - tn: 0.0000e+0

Epoch 9/100
707/707 [==============================] - 15s 21ms/step - loss: 0.0017 - tp: 22620.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00 - prc: 1.0000 - val_loss: 17.7191 - val_tp: 4042.0000 - val_fp: 45160.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.0822 - val_precision: 0.0822 - val_recall: 1.0000 - val_auc: 0.5015 - val_prc: 0.0824
Epoch 10/100
707/707 [==============================] - 15s 21ms/step - loss: 0.0014 - tp: 22620.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00 - prc: 1.0000 - val_loss: 18.8893 - val_tp: 4042.0000 - val_fp: 45160.0000 - val_tn: 0.0000e+00 - val_fn: 0.0000e+00 - val_accuracy: 0.0822 - val_precision: 0.0822 - val_recall: 1.0000 - val_auc: 0.5012 - val_prc: 0.0823
Epoch 11/100
707/707 [==============================] - 11s 16ms/step - loss: 0.0012 - tp: 22620.0000 - f

Epoch 4/100
707/707 [==============================] - 11s 16ms/step - loss: 0.0093 - tp: 22619.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 1.0000 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00 - prc: 1.0000 - val_loss: 10.9239 - val_tp: 4042.0000 - val_fp: 45147.0000 - val_tn: 13.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.0824 - val_precision: 0.0822 - val_recall: 1.0000 - val_auc: 0.5173 - val_prc: 0.0849
Epoch 5/100
707/707 [==============================] - 11s 15ms/step - loss: 0.0056 - tp: 22619.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 1.0000 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00 - prc: 1.0000 - val_loss: 12.3015 - val_tp: 4042.0000 - val_fp: 45151.0000 - val_tn: 9.0000 - val_fn: 0.0000e+00 - val_accuracy: 0.0823 - val_precision: 0.0822 - val_recall: 1.0000 - val_auc: 0.5096 - val_prc: 0.0836
Epoch 6/100
707/707 [==============================] - 11s 15ms/step - loss: 0.0047 - tp: 22620.0000 - fp: 0.0000e+00 - t

Epoch 11/100
707/707 [==============================] - 11s 16ms/step - loss: 0.0020 - tp: 22620.0000 - fp: 0.0000e+00 - tn: 0.0000e+00 - fn: 0.0000e+00 - accuracy: 1.0000 - precision: 1.0000 - recall: 1.0000 - auc: 0.0000e+00 - prc: 1.0000 - val_loss: 17.4385 - val_tp: 4036.0000 - val_fp: 45139.0000 - val_tn: 21.0000 - val_fn: 6.0000 - val_accuracy: 0.0825 - val_precision: 0.0821 - val_recall: 0.9985 - val_auc: 0.5024 - val_prc: 0.0825
Epoch 00011: early stopping
14 -------------------------------------


IndexError: list index out of range